## STL hyper-parameters tuning

In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Initialize PyJNIus  
import jnius_config
jnius_config.add_options('-Xrs','-Xmx4096m')
jnius_config.set_classpath('.', 'stl-decomp-4j.jar')
import jnius
from jnius import autoclass

# Standard Imports and Extensions 
%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (16.0, 10.0) # (24.0, 15.0)

import matplotlib.pyplot as plt
import pandas as pd

from IPython.core.display import display as core_display, HTML
core_display(HTML("<style>.container { width:100% !important; }</style>"))

# Load the Java STL classes

SeasonalTrendLoess = autoclass('com.github.servicenow.ds.stats.stl.SeasonalTrendLoess')
StlBuilder = autoclass('com.github.servicenow.ds.stats.stl.SeasonalTrendLoess$Builder')
CyclicSubSeriesSmoother = autoclass('com.github.servicenow.ds.stats.stl.CyclicSubSeriesSmoother')
SmoothBuilder = autoclass('com.github.servicenow.ds.stats.stl.CyclicSubSeriesSmoother$Builder')

#### Load Data

In [2]:
# Load the CO2 Data  

time = pd.date_range('3/1/1958 00:00:00', periods=708, freq='M')
df = pd.read_csv("co2.csv")
df.index = time
del df['row']
del df['time_years']
df.columns = ["data"]
values = df.data.values.tolist() # Unfortunately, PyJNIus doesn't seem to know about numpy

scoreFrequency_and_numberOfPointsPerPeriond = {'Daily': 7, 
                                               'Weekly': 13, 
                                               'Bi Weekly':6, 
                                               'Four Weekly':13, 
                                               'Monthly':12, 
                                               'Bi Monthly':6,
                                               'Quarterly': 4,
                                               'Fiscal Quarterly': 4,
                                               'Half Yearly': 2,
                                               'Yearly': 4,
                                               'Fiscal Yearly': 4}

#### Decompose time series and plot results

In [3]:
@interact(period_length=scoreFrequency_and_numberOfPointsPerPeriond, seasonal_width=(1,100,1))
def decompse_timeseries_using_STL(period_length=12, seasonal_width=35):
    # Build the smoother and perform the decomposition
    builder = StlBuilder()

    smoother = builder. \
        setPeriodLength(period_length). \
        setSeasonalWidth(seasonal_width). \
        setNonRobust(). \
        buildSmoother(values)

    stl = smoother.decompose()
    seasonal = stl.getSeasonal()
    trend = stl.getTrend()
    residual = stl.getResidual()
    
    # Plot the decomposition

    fig, axes = plt.subplots(4, sharex=True)

    _ = axes[0].plot(df)
    axes[0].set_title('data')
    _ = axes[1].plot(df.index, trend)
    axes[1].set_title('trend')
    _ = axes[2].plot(df.index, seasonal)
    axes[2].set_title('seasonal')
    _ = axes[3].plot(df.index, residual)
    axes[3].set_title('residual')

A Jupyter Widget

### Split data to training and testing sets

In [4]:
def split_train_test(period_length, n_periods):
    test = values[-(period_length * n_periods):]
    train = values[0:len(values)-len(test)]
    return train, test

### Forecast and plot results

In [5]:
def calculate_default_trend_width(periodicity, seasonal_width):
    return int(1.5 * periodicity / (1 - 1.5 / seasonal_width) + 0.5);
    

def extrapolate_trend(trend, number_of_periods, seasonal_width, period_length):
    trend_width = calculate_default_trend_width(period_length, seasonal_width)
    smooth_builder = SmoothBuilder()    
    smoother = smooth_builder \
    .setWidth(trend_width) \
    .setDataLength(len(trend)) \
    .setPeriodicity(period_length) \
    .setNumPeriodsForward(number_of_periods) \
    .setNumPeriodsBackward(0) \
    .build();
        
    extended_trend_length = len(trend) + number_of_periods * period_length
    extended_trend = smoother.smoothSeasonal(trend, extended_trend_length);
    
    return extended_trend;


@interact(period_length=scoreFrequency_and_numberOfPointsPerPeriond, seasonal_width=(1,60,1), number_of_periods=(1,2,1))
def forecast_using_STL_and_CyclicSubSeriesSmoother(period_length=12, seasonal_width=35, number_of_periods=1):
    
    train, test = split_train_test(period_length, number_of_periods)
    
    # Build the smoother and perform the decomposition
    builder = StlBuilder()

    smoother = builder. \
        setPeriodLength(period_length). \
        setSeasonalWidth(seasonal_width). \
        setNonRobust(). \
        buildSmoother(train)

    stl = smoother.decompose()
    seasonal = stl.getSeasonal()
    trend = stl.getTrend()
    residual = stl.getResidual()
    trend_plus_residual = [t + r for t, r in zip(trend, residual)]
    
    extended_trend = extrapolate_trend(trend_plus_residual, number_of_periods, seasonal_width, period_length)
    
    forecast = []
    forecasting_length = period_length * number_of_periods
    
    extrapolated_trend_values = extended_trend[len(train):len(extended_trend)]
    last_observed_seasonal_values = seasonal[(len(seasonal) - period_length):len(seasonal)]
    
    for i in range(0, forecasting_length):
        forecast.append(extrapolated_trend_values[i] + last_observed_seasonal_values[i % period_length])
    
    def plot_forecast():  
        forecast_x_axis = [i + len(train) for i in range(0, len(forecast))]
        fig1 = plt.figure()
        ax1 = fig1.add_subplot(211)
        ax1.plot(values)  
        ax2 = fig1.add_subplot(211)
        ax2.plot(forecast_x_axis, forecast)
        ax2.set_title('Full dataset with the forecasted values')
        
        ax3 = fig1.add_subplot(212)
        ax3.plot(test)  
        ax4 = fig1.add_subplot(212)
        ax4.plot(forecast)
        ax4.set_title('Testing dataset with the forecasted values')
        
    plot_forecast()
    

A Jupyter Widget